<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [0]:
!wget https://www.gutenberg.org/files/100/100-0.txt

--2019-12-16 23:37:41--  https://www.gutenberg.org/files/100/100-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5777367 (5.5M) [text/plain]
Saving to: ‘100-0.txt.2’

100-0.txt.2         100%[===================>]   5.51M  4.69MB/s    in 1.2s    

2019-12-16 23:37:42 (4.69 MB/s) - ‘100-0.txt.2’ saved [5777367/5777367]



In [0]:
data = []

with open('100-0.txt', 'r') as f:
    data.append(f.read())

### Cleaning Data

In [0]:
data = data[0]

In [0]:
len(data)

5573152

In [0]:
# cutting off the first characters
data = data[2806:]
data = data[:3500000]

In [0]:
data[:1000]

'\n\nFrom fairest creatures we desire increase,\nThat thereby beauty’s rose might never die,\nBut as the riper should by time decease,\nHis tender heir might bear his memory:\nBut thou contracted to thine own bright eyes,\nFeed’st thy light’s flame with self-substantial fuel,\nMaking a famine where abundance lies,\nThy self thy foe, to thy sweet self too cruel:\nThou that art now the world’s fresh ornament,\nAnd only herald to the gaudy spring,\nWithin thine own bud buriest thy content,\nAnd, tender churl, mak’st waste in niggarding:\n  Pity the world, or else this glutton be,\n  To eat the world’s due, by the grave and thee.\n\n\n                    2\n\nWhen forty winters shall besiege thy brow,\nAnd dig deep trenches in thy beauty’s field,\nThy youth’s proud livery so gazed on now,\nWill be a tattered weed of small worth held:\nThen being asked, where all thy beauty lies,\nWhere all the treasure of thy lusty days;\nTo say, within thine own deep sunken eyes,\nWere an all-eating shame

In [0]:
data = data.replace('\n', ' ')

In [0]:
# Replacing excess spaces with one space
data = data.replace('                       ', ' ')
data = data.replace('  ', ' ')

In [0]:
data[:1000]

' From fairest creatures we desire increase, That thereby beauty’s rose might never die, But as the riper should by time decease, His tender heir might bear his memory: But thou contracted to thine own bright eyes, Feed’st thy light’s flame with self-substantial fuel, Making a famine where abundance lies, Thy self thy foe, to thy sweet self too cruel: Thou that art now the world’s fresh ornament, And only herald to the gaudy spring, Within thine own bud buriest thy content, And, tender churl, mak’st waste in niggarding:  Pity the world, or else this glutton be,  To eat the world’s due, by the grave and thee. 2 When forty winters shall besiege thy brow, And dig deep trenches in thy beauty’s field, Thy youth’s proud livery so gazed on now, Will be a tattered weed of small worth held: Then being asked, where all thy beauty lies, Where all the treasure of thy lusty days; To say, within thine own deep sunken eyes, Were an all-eating shame, and thriftless praise. How much more praise deserv’

In [0]:
import re
import string
data = re.sub(r'[^a-zA-Z ^0-9]', '', data)

# Removing this for a moment to see what happens

In [0]:
chars = list(set(data))

In [0]:
len(chars)

63

In [0]:
# encode data as characters(chars)
char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

In [0]:
# creates the sequence data

#The length we want to analyze
maxlen = 28
# moving the target out by this many steps
# Each step is the training case for the next character
step = 5

encoded = [char_int[c] for c in data]

# reminder that our 'target' is next characters
sequences = []
next_chars = []

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i: i + maxlen])
    next_chars.append(encoded[i + maxlen])
    
print('sequences', len(sequences))

sequences 641215


In [0]:
import numpy as np

In [0]:
def char_encode_generator():
    for i in range(0, len(encoded) - maxlen, step):
        yield encoded[i: i + maxlen], encoded[i + maxlen]

In [0]:
# specifying x and y

x = np.zeros((len(sequences), maxlen, len(chars)))
y = np.zeros((len(sequences), len(chars)))

In [0]:
for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_chars[i]] = 1

In [0]:
x.shape

(641215, 28, 63)

In [0]:
"""
We can do more that classification and regression,
We can generate data!
"""

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

"""
Using numpy plus some systems things for... [jon-cody trails off...]"""
import numpy as np
import random
import sys
import os

In [0]:
# Build the model
model = Sequential()
model.add(LSTM(128,input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = np.random.randint(0, len(data) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = data[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_int[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = int_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [0]:
model.fit(x, y,
          batch_size=450,
          epochs=5,
          callbacks=[print_callback])

Train on 641215 samples
Epoch 1/5
640800/641215 [============================>.] - ETA: 0s - loss: 1.9000
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "se wars his great opposer Co"
se wars his great opposer Come the sir the not to you the not   The great the count                                                                                                                                                                                                                                                                                                                                                         
----- diversity: 0.5
----- Generating with seed: "se wars his great opposer Co"
se wars his great opposer Come the Don her can my lord of the wild best the carent Well not so not one and the note   Where no prove you I sain the wind me and here stould to that not me in the our price and him say the bast to be that thy was the not so me for lord a

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN